In [1]:
# General imports
import numpy as np
import torch
import matplotlib.pyplot as plt
# DeepMoD stuff
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold, PDEFIND
from deepymod.training import train
from deepymod.training.sparsity_scheduler import TrainTestPeriodic
from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Making dataset

noise = 0.02
A = 1
v = 0.25

runs = 1
dataset = Dataset(BurgersDelta, A=A, v=v)
n_x = 10
x = np.linspace(-2, 2, n_x)
t = np.linspace(0.1, 1.1, 100) 
t_grid, x_grid = np.meshgrid(t, x, indexing='ij')
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=0, noise=noise, random=True, normalize=False)


In [6]:
X.shape

torch.Size([1000, 2])

In [3]:
X, y = X.to(device), y.to(device)

In [4]:
for run in np.arange(runs):
    network = NN(2, [30, 30, 30, 30], 1)
    library = Library1D(poly_order=2, diff_order=3) # Library function
    estimator = Threshold(0.2) # Sparse estimator 
    constraint = LeastSquares() # How to constrain
    model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

    sparsity_scheduler = TrainTestPeriodic(periodicity=50, patience=500, delta=1e-7) # in terms of write iterations
    optimizer = torch.optim.Adam(model.parameters(), betas=(0.99, 0.99), amsgrad=True, lr=1e-3) # Defining optimizer

    train(model, X, y, optimizer, sparsity_scheduler, log_dir='runs/test_large_wait_1', split=0.8, write_iterations=25, max_iterations=50000, delta=1e-4, patience=500) 
    print(model.constraint_coeffs(sparse=True, scaled=True))

 26950  MSE: 3.62e-05  Reg: 4.16e-06  L1: 3.36e+00 Algorithm converged. Writing model to disk.
[tensor([[ 0.0034],
        [-0.0242],
        [ 0.7797],
        [-0.0652],
        [ 0.0308],
        [-0.3031],
        [-0.1884],
        [ 0.4757],
        [-0.0945],
        [-0.0565],
        [ 0.5018],
        [-0.8378]], device='cuda:0', grad_fn=<MulBackward0>)]
